In [16]:
from symred.unitary_partitioning import AntiCommutingOp

from symred.symplectic_form import PauliwordOp
import numpy as np
from typing import Dict, List, Union

In [7]:
test = AntiCommutingOp({'XXY':1, 'XZY':2, 'ZYZ':1, 'IIX':1})
rots = test.gen_LCU()

print(rots)

0.8300495386+0.0000000000j III +
-0.0000000000-0.2276758527j ZYY +
0.0000000000+0.4553517055j XZZ +
0.0000000000+0.2276758527j XXZ


In [3]:
%timeit one = (rots * test).cleanup_zeros()

808 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
one = (rots * test).cleanup_zeros()
%timeit two = (one * rots.conjugate).cleanup_zeros()

814 µs ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit test.gen_seq_rotations()

2.07 ms ± 40.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
test = AntiCommutingOp({'XXY':1, 'XZY':2, 'IIZ':1})
rots = test.gen_seq_rotations()

rots_working = test.X_sk_rotations

In [25]:
class AntiCommutingOp2(PauliwordOp):


    def __init__(self,
                 AC_operator: Union[List[str], Dict[str, float], np.array],
                 coeff_list: Union[List[complex], np.array] = None):
        super().__init__(AC_operator, coeff_list)

        # check all operators anticommute
        anti_comm_check = self.adjacency_matrix.astype(int) - np.eye(self.adjacency_matrix.shape[0])
        assert(np.einsum('ij->', anti_comm_check) == 0), 'operator needs to be made of anti-commuting Pauli operators'

        # normalization factor
        self.gamma_l = np.linalg.norm(self.coeff_vec)
        # normalize coefficients
        self.coeff_vec = self.coeff_vec/self.gamma_l

    def lexicographical_sort(self):
        """
        sort object into lexicographical order

        Returns:

        """
        # convert sym form to list of ints
        int_list = self.symp_matrix @ (1 << np.arange(self.symp_matrix.shape[1], dtype=object)[::-1])
        lex_ordered_indices = np.argsort(int_list)
        return lex_ordered_indices


    def get_lowest_dense_index(self):

        # np.logical_or(X_block, Z_block)
        pos_terms_occur = np.logical_or(self.symp_matrix[:, :self.n_qubits],self.symp_matrix[:, self.n_qubits:])

        int_list = pos_terms_occur @ (1 << np.arange(pos_terms_occur.shape[1])[::-1])
        s_index = np.argmin(int_list)
        return s_index


    def gen_sequence_of_rotations(self, s_index=None):
        """

        Args:
            s_index:

        Returns:

        """
        X_sk_theta_sk_list=[]
        if self.n_terms == 1:
            return None

        s_index=0
        # if s_index is None:
        #     s_index = self.get_lowest_dense_index()

        # take β_s P_s
        P_s = PauliwordOp(self.symp_matrix[s_index], [1])
        β_s = self.coeff_vec[s_index]

        # then remove from symp mat and coeff vec
        symp_matrix_no_Ps = np.delete(self.symp_matrix, s_index, axis=0)
        coeff_vec_no_βs = np.delete(self.coeff_vec, s_index, axis=0)

        theta_sk = np.arctan(coeff_vec_no_βs[0] / β_s)
        if β_s.real < 0:
            theta_sk = theta_sk + np.pi
        assert(np.isclose((coeff_vec_no_βs[0] * np.cos(theta_sk) - β_s * np.sin(theta_sk)), 0)), 'term not zeroing out'

        # X_sk = 1j * Ps @ Pk
        X_sk = P_s * PauliwordOp(symp_matrix_no_Ps[0], [-1j])
        X_sk_theta_sk_list.append((X_sk, theta_sk))


        # β_s_new = np.sqrt(coeff_vec_no_βs[0] ** 2 + β_s ** 2)
        β_s_new = np.linalg.norm([coeff_vec_no_βs[0],  β_s])
        for ind, Pk in enumerate(symp_matrix_no_Ps[1:]):
            β_k = coeff_vec_no_βs[1:][ind]

            # X_sk = 1j * Ps @ Pk
            theta_sk = np.arctan(β_k / β_s_new)
            assert (np.isclose((β_k * np.cos(theta_sk) - β_s_new * np.sin(theta_sk)), 0)), 'term not zeroing out'

            # X_sk = 1j * Ps @ Pk
            X_sk = P_s * PauliwordOp(Pk, [-1j])
            X_sk_theta_sk_list.append((X_sk, theta_sk))

            β_s_new = np.linalg.norm([β_k, β_s])

        return X_sk_theta_sk_list


    def Apply_SeqRot(self, list_rotations):
        AC_op_rotated = PauliwordOp(self.symp_matrix, self.coeff_vec)
        for X_sk, theta_sk in list_rotations:
            if X_sk.coeff_vec[0].real<0:
                AC_op_rotated = AC_op_rotated._rotate_by_single_Pword(X_sk, theta_sk).cleanup_zeros()
        return AC_op_rotated.cleanup_zeros()

In [29]:
test = AntiCommutingOp2({'XXY':1, 'XZY':2, 'IIZ':1})
rots = test.gen_sequence_of_rotations()
for r, ang in rots:
    print(r, ang)

-1.0000000000+0.0000000000j IYI (1.1071487177940904+0j)
1.0000000000-0.0000000000j XXX (0.42053433528396517+0j)


In [32]:
for r, ang in rots_working:
    print(r, ang)

1.0000000000-0.0000000000j XZX (-1.1071487177940904+0j)
1.0000000000-0.0000000000j XXX (-0.42053433528396517+0j)


In [27]:
%timeit test.gen_sequence_of_rotations()

402 µs ± 5.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
